In [1]:
# %env MINERL_DATA_ROOT=/Volumes/CORSAIR/data

In [2]:
import minerl  # NOTE: we need gym>=0.13.1,<0.20
import gym
from train import get_agent, get_dynamics_environment, FMC, get_data_handler
from fgz.training.fgz_trainer import FGZTrainer
import torch
import wandb

/home/nollyd/anaconda3/envs/minerl/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
torch.cuda.empty_cache()  # fix memory leaks

In [4]:
minerl_env = gym.make('MineRLBasaltBuildVillageHouse-v0')
agent = get_agent()
dynamics_env = get_dynamics_environment(minerl_env)

Loading model data/VPT-models/foundation-model-1x.model
with weights data/VPT-models/foundation-model-1x.weights


In [ ]:
agent.device

In [ ]:
# dynamics_env.batched_action_space_sample()

In [ ]:
# dummy_initial_state = torch.ones(4096, dtype=float)
# dynamics_env.set_all_states(dummy_initial_state)

In [ ]:
data_handler = get_data_handler(agent)
str(data_handler.loaders[0].trajectories[0])

In [ ]:
# t = data_handler.sample_single_trajectory()
# for window in t:
#     print(len(window))
#     for frame, state_embedding, action in window:
#         print(state_embedding.shape)
#         break
#     break

In [ ]:
dynamics_function_optimizer = torch.optim.Adam(
    dynamics_env.dynamics_function.parameters(),
    lr=0.0002,
    weight_decay=1e-4,
)

In [ ]:
use_wandb = True

fmc = FMC(dynamics_env, freeze_best=True)
trainer = FGZTrainer(agent, fmc, data_handler, dynamics_function_optimizer, unroll_steps=8, use_wandb=use_wandb)

if use_wandb:
    wandb.init(project="fgz-v0")

In [ ]:
# trainer.save("test_trainer_save.pth")
# loaded_trainer = FGZTrainer.load("test_trainer_save.pth", agent)
# loaded_trainer.evaluate("MineRLBasaltMakeWaterfall-v0", render=True, max_steps=4096, force_no_escape=True)

In [ ]:
# trainer.evaluate("MineRLBasaltMakeWaterfall-v0", render=True, max_steps=4096, force_no_escape=True)

In [ ]:
# trainer.eval_actions

In [ ]:
train_steps = 1000
for _ in range(train_steps):
    trainer.train_sub_trajectory(use_tqdm=True)

In [ ]:
trainer.save("fgz_dynamics_trained.pth")

In [5]:
trainer = FGZTrainer.load("fgz_dynamics_trained.pth", agent)

In [6]:
trainer.evaluate("MineRLBasaltMakeWaterfall-v0", render=True, max_steps=4096, force_no_escape=True)

2022-10-09 23:21:45 nollyd minerl.env.malmo.instance.3d4e4e[808347] INFO Starting Minecraft process: ['/home/nollyd/anaconda3/envs/minerl/lib/python3.10/site-packages/minerl/env/../MCP-Reborn/launchClient.sh', '-port', '12784', '-env', '-runDir', '/home/nollyd/anaconda3/envs/minerl/lib/python3.10/site-packages/minerl/env/../MCP-Reborn/run']
2022-10-09 23:21:45 nollyd process_watcher[808347] INFO Launching process watcher daemonizer.
/home/nollyd/anaconda3/envs/minerl/lib/python3.10/runpy.py:126: RuntimeWarning: 'minerl.utils.process_watcher' found in sys.modules after import of package 'minerl.utils', but prior to execution of 'minerl.utils.process_watcher'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2022-10-09 23:21:46 nollyd process_watcher[808347] INFO Process watcher daemonizer launched successfully.
2022-10-09 23:21:46 nollyd minerl.env.malmo.instance.3d4e4e[808347] DEBUG /home/nollyd/anaconda3/envs/minerl/lib/python3.10/site-packages/minerl/env/../MCP-